In [1]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.0.9000     ✔ purrr   0.3.4     
✔ tibble  3.0.1          ✔ dplyr   0.8.5     
✔ tidyr   1.0.3          ✔ stringr 1.4.0     
✔ readr   1.3.1          ✔ forcats 0.5.0     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



[1] "league-of-legends-diamond-ranked-games-10-min"

In [2]:
data = read.csv("../input/league-of-legends-diamond-ranked-games-10-min//high_diamond_ranked_10min.csv")
str(data)

'data.frame':	9879 obs. of  40 variables:
 $ gameId                      : num  4.52e+09 4.52e+09 4.52e+09 4.52e+09 4.44e+09 ...
 $ blueWins                    : int  0 0 0 0 0 1 1 0 0 1 ...
 $ blueWardsPlaced             : int  28 12 15 43 75 18 18 16 16 13 ...
 $ blueWardsDestroyed          : int  2 1 0 1 4 0 3 2 3 1 ...
 $ blueFirstBlood              : int  1 0 0 0 0 0 1 0 0 1 ...
 $ blueKills                   : int  9 5 7 4 6 5 7 5 7 4 ...
 $ blueDeaths                  : int  6 5 11 5 6 3 6 13 7 5 ...
 $ blueAssists                 : int  11 5 4 5 6 6 7 3 8 5 ...
 $ blueEliteMonsters           : int  0 0 1 1 0 1 1 0 0 1 ...
 $ blueDragons                 : int  0 0 1 0 0 1 1 0 0 1 ...
 $ blueHeralds                 : int  0 0 0 1 0 0 0 0 0 0 ...
 $ blueTowersDestroyed         : int  0 0 0 0 0 0 0 0 0 0 ...
 $ blueTotalGold               : int  17210 14712 16113 15157 16400 15899 16874 15305 16401 15057 ...
 $ blueAvgLevel                : num  6.6 6.6 6.4 7 7 7 6.8 6.4 7.2 6.8 ..

In [7]:
library(caret)

train_index = createDataPartition(data$blueWins, list=FALSE)

x = data  %>% select(-c(gameId, blueWins))
x = scale(x, center=TRUE)
y = data  %>% select(c(blueWins))

scaled_data = cbind(x, y)
train_data = scaled_data[ train_index, ]
test_data  = scaled_data[-train_index, ]

In [11]:
library(MASS)

logistic_model = glm(as.factor(blueWins) ~ ., data=scaled_data, family='binomial')
final_model = stepAIC(logistic_model, direction='both', trace=FALSE)
summary(final_model)


Call:
glm(formula = as.factor(blueWins) ~ blueWardsPlaced + blueFirstBlood + 
    blueKills + blueDeaths + blueDragons + blueTowersDestroyed + 
    blueTotalMinionsKilled + blueGoldDiff + blueExperienceDiff + 
    redEliteMonsters + redDragons + redTowersDestroyed + redTotalMinionsKilled + 
    redTotalJungleMinionsKilled, family = "binomial", data = scaled_data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.6989  -0.8709  -0.1405   0.8691   2.7706  

Coefficients:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 -0.002338   0.024056  -0.097 0.922576    
blueWardsPlaced             -0.039442   0.023892  -1.651 0.098772 .  
blueFirstBlood               0.041018   0.026079   1.573 0.115757    
blueKills                   -0.142402   0.060790  -2.343 0.019153 *  
blueDeaths                   0.139305   0.062180   2.240 0.025068 *  
blueDragons                  0.176029   0.030940   5.689 1.27e-08 ***
blueTowersDest

In [12]:
pred = predict(logistic_model, test_data, type="response")
pred = ifelse(pred >= 0.5, 1, 0)
confusionMatrix(as.factor(as.vector(pred)), as.factor(test_data$blueWins))

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”


Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 1822  674
         1  663 1780
                                          
               Accuracy : 0.7293          
                 95% CI : (0.7167, 0.7417)
    No Information Rate : 0.5031          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.4586          
                                          
 Mcnemar's Test P-Value : 0.7845          
                                          
            Sensitivity : 0.7332          
            Specificity : 0.7253          
         Pos Pred Value : 0.7300          
         Neg Pred Value : 0.7286          
             Prevalence : 0.5031          
         Detection Rate : 0.3689          
   Detection Prevalence : 0.5054          
      Balanced Accuracy : 0.7293          
                                          
       'Positive' Class : 0               
                        